In [ ]:
# Imports
import numpy as np
from numpy import linalg as LA

# Plotting imports
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D


import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import math


import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

In [ ]:
data = np.loadtxt( "ANNdata.txt" )
dataset = pd.DataFrame({'alpha': data[:,0], 'tU': data[:,1] , 'tS': data[:,2], 'norm': data[:,3]})
dataset.tail()

In [ ]:
plt.plot(data[:,0]*180/np.pi, data[:,3], color='black')
plt.xlabel('Cone angle (deg)',fontsize=13)
plt.ylabel('Euclidean-Norm (-)',fontsize=13)
plt.tick_params(axis='both', labelsize=13)
plt.grid()

In [ ]:
plt.plot(data[:,1]*5022415/(365.25*24*3600), data[:,3], color='black')
plt.xlabel('TOF Unstable (years)',fontsize=13)
plt.ylabel('Euclidean-Norm (-)',fontsize=13)
plt.tick_params(axis='both', labelsize=13)
plt.grid()

In [ ]:
plt.plot(-data[:,2]*5022415/(365.25*24*3600), data[:,3], color='black')
plt.xlabel('TOF Stable (years)',fontsize=13)
plt.ylabel('Euclidean-Norm (-)',fontsize=13)
plt.tick_params(axis='both', labelsize=13)
plt.grid()

In [ ]:
inp_train_dataset = dataset.sample(frac=0.8,random_state=23)
out_train_dataset = inp_train_dataset.pop('norm')

inp_test_dataset = dataset.drop(inp_train_dataset.index)
out_test_dataset = inp_test_dataset.pop('norm')

In [ ]:
inp_train_stats = inp_test_dataset.describe()
inp_train_stats = inp_train_stats.transpose()
inp_train_stats

In [ ]:
def norm(x):
  return (x - inp_train_stats['mean']) / inp_train_stats['std']

inp_normed_train_data = norm(inp_train_dataset)
inp_normed_test_data = norm(inp_test_dataset)

In [ ]:
inp_normed_train_data

In [ ]:
def build_model():
  model = keras.Sequential([
    layers.Dense(50, activation='relu', input_shape=[3]),
    layers.Dense(50, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer='adam',
                metrics=['mae', 'mse'])
  return model

In [ ]:
model = build_model()

In [ ]:
EPOCHS = 100

history = model.fit(
  x = inp_normed_train_data, y = out_train_dataset,
  epochs=EPOCHS, validation_split = 0.2, verbose = 0)

plt.plot(history.history['mean_absolute_error'])
plt.plot(history.history['val_mean_absolute_error'])
plt.title('Mean absolute error')
plt.xlabel('epoch')
plt.legend(['MAE train','MAE val.'],loc='upper right',fontsize=13)
plt.show()

In [ ]:
history.history['val_mean_absolute_error'][-1]

In [ ]:
test_predictions = model.predict(inp_normed_test_data)
test_labels = out_test_dataset

In [ ]:
a = plt.axes(aspect='equal')
plt.scatter(out_test_dataset, test_predictions, color='black')
plt.xlabel('Euclidean-Norm True Values',fontsize=13)
plt.ylabel('Euclidean-Norm Predictions',fontsize=13)
plt.tick_params(axis='both', labelsize=13)
plt.grid()

lims = [0, 2]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims, color='black')


In [ ]:
test_predictions = model.predict(inp_normed_test_data)
test_labels = out_test_dataset
test_labels = np.array([np.array(test_labels)]).T

error = np.array(test_predictions) - test_labels

plt.hist(error, bins = 25, color='black')
plt.xlabel("Prediction Error",fontsize=13)
_ = plt.ylabel("Count",fontsize=13)
plt.tick_params(axis='both', labelsize=13)
plt.grid()

error_mean = np.mean(error)
error_stdv = np.std(error)

error_mean
error_stdv

In [ ]:
MCruns = 100000;
np.random.seed(123)
alphas = np.random.uniform(-55*np.pi/180, 55*np.pi/180, MCruns).tolist()
tUs    = np.random.uniform(0, 35, MCruns).tolist()
tSs    = np.random.uniform(-35, 0, MCruns).tolist()

inp_predict_data = pd.DataFrame({'alpha': alphas, 'tU': tUs , 'tS': tSs})

inp_normed_predict_data = norm(inp_predict_data)

model_predictions = model.predict(inp_normed_predict_data)


In [ ]:
plt.scatter(inp_predict_data['alpha']*180/np.pi, model_predictions, color='black')
plt.xlabel('Cone angle (deg)',fontsize=13)
plt.ylabel('Euclidean-Norm (-)',fontsize=13)
plt.tick_params(axis='both', labelsize=13)
plt.grid()

In [ ]:
# Cleaning Plot
alphas_plot[np.argmin(eucl_plot)]
alphas = np.round(np.array(alphas)*180/np.pi,1)
alphas_sorted = np.sort(alphas)
alphas_arg = np.argsort(alphas)
alphas_plot = [alphas_sorted[0]]
eucl_plot = [model_predictions[alphas_arg[0]]]
eucl_min = model_predictions[alphas_arg[0]]
for i in range(len(alphas)-1):
    if alphas_sorted[i+1]>alphas_sorted[i]:
        alphas_plot.append(alphas_sorted[i])
        eucl_plot.append(eucl_min)
        eucl_min = np.Inf
    if model_predictions[alphas_arg[i+1]] < eucl_min:
        eucl_min = model_predictions[alphas_arg[i+1]]

In [ ]:
plt.plot(alphas_plot, eucl_plot, color='black')
plt.xlabel('Cone angle (deg)',fontsize=13)
plt.ylabel('Euclidean-Norm (-)',fontsize=13)
plt.tick_params(axis='both', labelsize=13)
plt.grid()
alphas_plot[np.argmin(eucl_plot)]
# min(eucl_plot)

In [ ]:
# %matplotlib qt

%matplotlib inline